# Exp 06: Hard code NumPy data types

In [24]:
# standard libraries
from time import perf_counter_ns
import time

In [25]:
# external libraries
from matplotlib.patches import Rectangle
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import numpy as np
import pandas as pd
import seaborn as sns

# custom libraries
from _run_constants import *
from part_00_file_db_utils import *
from part_00_process_functions import *

In [26]:
word_df, wg_df, letter_dict, char_matrix, \
    word_group_id_list, word_id_list, wchar_matrix = load_input_data(
        db_path=rc.DB_PATH, db_name=rc.DB_NAME,
        in_file_path=rc.IN_FILE_PATH)

...loading words into a dataframe...
...query execution took: 1.6 seconds...
...loading word groups into a dataframe...
...query execution took: 1.37 seconds...
...loading the letter dictionary...
...loading the char matrix...
...subsetting the char matrix...


In [27]:
col_names = ['letter_selector_mod', 'n_records']
ls_df = build_letter_selector_df(df = wg_df, ls_nchar=3,                          
                                 letter_selector_col_name='letter_selector',
                                 letter_selector_id_col_name='letter_selector_id')

In [28]:
# function to return the index position of each letter
def get_ls_index(ls:str):
    return [letter_dict[l] for l in ls]

In [29]:
ls_df['ls_index'] = ls_df['letter_selector'].map(get_ls_index)

In [30]:
ls_df['ls_nchar'].value_counts()

ls_nchar
3    2250
2     111
1      26
Name: count, dtype: int64

In [31]:
# load the total number of anagrams
n_possible_anagrams = load_possible_anagrams(db_path=rc.DB_PATH,
                                             db_name=rc.DB_NAME)

...query execution took: 0.0 seconds...


In [32]:
ls_df = ls_df.reset_index(drop = True)

In [33]:
col_names = ['letter_selector', 'letter_selector_id']
wg_df = pd.merge(left = wg_df, right = ls_df[col_names])

In [34]:
wg_df.head()

,word,lcase,n_chars,first_letter,word_id,word_group_id,letter_group,letter_group_ranked,word_group_count,letter_selector,n_records,letter_selector_id
0,A,a,1,a,0,0,a,a,1,a,1,0
1,aa,aa,2,a,1,1,a,a,1,a,1,0
2,aal,aal,3,a,2,2,al,la,2,la,1,1081
3,aalii,aalii,5,a,3,3,ail,lai,1,lai,1,1083
4,aam,aam,3,a,4,4,am,ma,2,ma,1,1114


In [35]:
wg_df['letter_selector'].unique().shape

(2387,)

In [36]:
# get clever with the NumPy DataTypes
print(np.iinfo(np.int8))
print(np.iinfo(np.int16))
print(np.iinfo(np.int32))
print(np.iinfo(np.int64))

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



In [37]:
print(wchar_matrix.dtype)
print(wchar_matrix.max())

int32
8


In [38]:
print(word_group_id_list.dtype)
print(word_group_id_list.max())

int64
215841


In [39]:
# convert the wchar_matrix to int8 
# convert the word_group_id_list to int32

In [40]:
wchar_matrix = wchar_matrix.astype(np.int8)
word_group_id_list = word_group_id_list.astype(np.int32)

In [41]:
n_possible_anagrams / 2

99421122.5

In [42]:
temp_matrix = wg_df[['letter_selector_id', 'word_group_id']].to_numpy(dtype = np.int32)

In [47]:
# run it!
run_start_time=perf_counter_ns()
# create the output list
output_list = np.full(shape = (int(n_possible_anagrams / 2), 2), fill_value=-1,dtype=np.int32)
output_time_list = []

# start counting
anagram_pair_count = 0

for ls_row_id, ls_row in ls_df.iloc[:None].iterrows():    
    if ls_row_id % 100 == 0:
        print(ls_row_id)
    start_time = perf_counter_ns()
    
    # get letter selector id information    
    ls_id = ls_row['letter_selector_id']
    # this is a selector on a 26 column matrix
    ls_id_index = np.array(object = ls_row['ls_index'], dtype = np.int8)        
        
    ##
    # SUBSET THE wchar_matrix by column selector
    ##    
    outcome_indices = np.all(wchar_matrix[:, ls_id_index] >= 1, axis=1)
    #print(outcome_indices.dtype)
    
    # this is the sub-matrix from which to query
    ls_wchar_matrix = wchar_matrix[outcome_indices, :]
    #print(ls_wchar_matrix.dtype)
        
    # this is the list of word group ids that correspond to the word group ids
    # in the ls_wchar_matrix
    temp_wg_id_list = word_group_id_list[outcome_indices]
        
    # this is the number of word groups that meet certain criteria. 
    # for example, words that feature the letters: 'buc'    
    n_search_space = temp_wg_id_list.shape[0]        
    
    # the current list of words featuring the set of least common letters.
    # these are the words have the least common letters of 'bro'    
    #curr_wg_id_list = wg_df.loc[wg_df['letter_selector_id'] == ls_id, 'word_group_id'].to_numpy(dtype=np.int32)        
    curr_wg_id_list = temp_matrix[temp_matrix[:, 0] == ls_id, 1]
         
    for i_curr_wg_id, curr_wg_id in enumerate(curr_wg_id_list):    
            
        # get different word group ids?
        temp_wg_id = np.where(temp_wg_id_list == curr_wg_id)[0][0]
        
        #print(curr_wg_id, temp_wg_id)
        
        outcome_word_id_list = temp_wg_id_list[np.all(a = (ls_wchar_matrix - ls_wchar_matrix[temp_wg_id, :]) >= 0, axis = 1)]        
                        
        n_from_words = outcome_word_id_list.shape[0]
        
        if n_from_words > 0:
            outcome_word_id_list = format_output_list(outcome_word_id_list=outcome_word_id_list, wg_id=curr_wg_id)
            #print(outcome_word_id_list.dtype)
                        
            # enumerate the from/parent words
            new_anagram_pair_count = anagram_pair_count + n_from_words
            
            output_list[anagram_pair_count:new_anagram_pair_count, :] = outcome_word_id_list
            
            # update the anagram pair count
            anagram_pair_count = new_anagram_pair_count

    curr_time = calc_time(time_start=start_time, round_digits=8)
    output_time_list.append([ls_id, n_search_space, curr_time])

print('...time to find parent/child word relationships')
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)
print('...truncating output list...')
output_indices = np.all(output_list >= 0, axis=1)
output_list = output_list[output_indices,]
print(output_list.shape)
time_proc = calc_time(time_start=run_start_time, round_digits=4)
compute_elapsed_time(seconds=time_proc)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
...time to find parent/child word relationships
Hours: 0 | minutes: 1 | seconds: 32.4412
...truncating output list...
(73218235, 2)
Hours: 0 | minutes: 1 | seconds: 37.145


In [44]:
# c2
# current technique: 2 minutes, 37 seconds
# setting the numpy data types: 1 minute, 41 seconds.
# being even more strict with the data types: 1 minutes, 26 seconds

In [45]:
# approximately 19 seconds
from_word_counter = collections.Counter(output_list[:,1])
to_word_counter = collections.Counter(output_list[:,0])

In [46]:
# the number of from word groups: should be 26
print(from_word_counter[746]) # should be 26
print(to_word_counter[746]) # should be 329

0
2
